# Sychronisation de la copie de HAL
1. Identification des notices supprimées depuis la dernière synchro
    * interrogation en json pour récupérer tous les docid et HalID du jour
        * format du nom du fichier : Tous_Les_docids_du_jour_AAAA-MM-JJ.csv (exemple : Tous_Les_docids_du_jour_2024-08-02.csv)
    * comparaison avec la liste récupérée la veille
        * on ne compare pas les docid, mais les halID : lorsqu'une nouvelle version est créée, le docid de la version précédente disparaît de l'API, on peut donc croire que c'est une suppression. Si une notice est supprimée, son halID disparaît, toutes versions confondues. Donc on compare les halID. Si un halID n'est plus présent dans la liste du jour par rapport à la liste de la veille, c'est que la notice est supprimée.
        * le fichier de la veille aura le même format de nom. par exemple : Tous_Les_docids_du_jour_2024-08-01.csv (note : la première liste a été élaborée avec le script Premiere_copie_Hal_DATADCIS.ypnb)
        * Création d'un fichier des notices supprimées
        * nom du fichier : fichiers_a_supprimer.csv
    * Suppression de tout fichier dont le nom contient le halID, dans les dossiers contenant les notices xml et les fichiers associes
        * nom des dossiers : /data/Notices_Xml-tei_de_Hal et ../pdf_de_hal



2. Synchronisation de la copie de HAL
    (voir script Synchro_Deuxieme_Partie_HAL_DATADCIS)
            





## Synchronisation quotidienne : identification des notices à supprimer

In [1]:
###############################################################
## Récupération des docid pour comparaison avec la veille
## objectif : identifier les notices supprimées
################################################################
import requests
import time
import os
import pandas as pd
from datetime import datetime, timedelta
import logging
import sys



# Obtenir la date actuelle
date_extraction_current = datetime.now().strftime("%Y-%m-%d")

# Définition des variables
compteur = 0
Les_docids = []

## Spécifier le répertoire de log
log_directory = '/data/log/'
## Créer le répertoire s'il n'existe pas
os.makedirs(log_directory, exist_ok=True)

## Déterminer le chemin du dossier DocidDeHAL (au même niveau que le dossier courant)
docid_folder ='/data/DocidDeHAL'
# Créer le répertoire si nécessaire
os.makedirs(docid_folder, exist_ok=True)


# Construire le nom du fichier de log avec le chemin complet
log_file = os.path.join(log_directory, date_extraction_current + '_log_synchro_part1_hal.txt')

# Configuration du logger
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

#Définition du nom du fichier
fichier_des_docids_current = os.path.join(docid_folder,f'Tous_Les_docids_du_jour_{date_extraction_current}.csv')


# class StopExecution(Exception):
#     def _render_traceback_(self):
#         return []


# # Si le fichier existe déjà, sortir du script.
# if os.path.exists(fichier_des_docids_current):
#     logging.info(f"Le fichier {fichier_des_docids_current} existe déjà. Arrêt du script.")
#     print(f"Le fichier {fichier_des_docids_current} existe déjà. Arrêt du script.")
#     raise StopExecution

#### Processus de récupération de la liste des notices présentes dans HAL à la date du jour

# URL de base de l'API
base_url = "https://api.archives-ouvertes.fr/search"

query = "*"

# Paramètres de la requête
params = {
    "q": query,
    "wt": "json",
    "fl": "docid, halId_s,uri_s,submitType_s",
    "rows": 10000,
    "sort": "docid asc"
}

#url entière : https://api.archives-ouvertes.fr/search/INRIA2?q=*&rows=10000&wt=json&fl=docid,halId_s,uri_s,submitType_s&sort_dodicd asc


# Initialisation du cursorMark (qui permet de réitérer la requête jusqu'à la fin des réponses de l'API)
cursor_mark = "*"
previous_cursor_mark = None


while cursor_mark != previous_cursor_mark:
    # Mise à jour du cursorMark
    params["cursorMark"] = cursor_mark
    #print(f"CursorMark: {cursor_mark}")
    compteur += 1

    # Envoi de la requête GET
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        results_json = response.json()
        docs = results_json['response']['docs']
        Nbe_depots = results_json['response']['numFound']
        #print({Nbe_depots})
        for doc in docs:
            docid = doc['docid']
            halId = doc['halId_s']
            uri = doc['uri_s']
            submitType = doc['submitType_s']
            Les_docids.append([docid, halId, uri,submitType])
            #print(f"DocID: {docid}, URI: {uri}")

        next_cursor_mark = results_json.get('nextCursorMark')
        if not next_cursor_mark:
            break

    # Mise à jour du cursorMark pour la prochaine itération
    previous_cursor_mark = cursor_mark
    cursor_mark = next_cursor_mark

    #if compteur > 500:  # Limite arbitraire pour les tests
        #break

    # Pause pour éviter de surcharger l'API
    time.sleep(0.1)


# Création du DataFrame et exportation en CSV
df_current = pd.DataFrame(Les_docids, columns=['docID', 'halId', 'uri','submitType_s'])

df_current.to_csv(fichier_des_docids_current, index=False)

logging.info(f"Terminé.  {len(df_current)} notices trouvées.")
print(f"Terminé.  {len(df_current)} notices trouvées.")




KeyboardInterrupt: 

In [3]:
##################################################################
# Comparaison du fichier du jour et de celui de la veille
# Enregistrement du fichier des notices à supprimer
# On a aussi une liste de notices ajoutées, mais on n'utilise pas cette information pour l'instant.
##################################################################
import requests
import time
import os
import pandas as pd
from datetime import datetime, timedelta
import logging

print(f"étape de comparaison des listes quotidiennes")

# Obtenir la date actuelle
date_extraction_current = datetime.now().strftime("%Y-%m-%d")

# Obtenir la date de la veille
date_extraction_previous = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

## Déterminer le chemin du dossier DocidDeHAL (au même niveau que le dossier courant)
docid_folder ='/data/DocidDeHAL'
suppressed_folder ='/data/Suppressions'



# Créer le répertoire si nécessaire
os.makedirs(docid_folder, exist_ok=True)
os.makedirs(suppressed_folder, exist_ok=True)






#Redéfinition des noms des fichiers à comparer
fichier_des_docids_previous = os.path.join(docid_folder,f'Tous_Les_docids_du_jour_{date_extraction_previous}.csv')
fichier_des_docids_current = os.path.join(docid_folder,f'Tous_Les_docids_du_jour_{date_extraction_current}.csv')

# Infos pour le fichier de log
logging.info(f"date du jour : {date_extraction_current}")
logging.info(f"fichier du jour: {fichier_des_docids_current}")
logging.info(f"fichier de la veille: {fichier_des_docids_previous}")

# Lire les fichiers CSV dans des DataFrames
df_previous = pd.read_csv(fichier_des_docids_previous, dtype=str)
#df2 = df_current
df_current = pd.read_csv(fichier_des_docids_current, dtype=str)


# Créer une copie (exigence Notebook)
df_previous = df_previous.copy()  
df_current = df_current.copy()  

# Convertir les colonnes 'docID' en chaînes de caractères pour assurer la compatibilité
df_previous['halId'] = df_previous['halId'].astype(str)
df_current['halId'] = df_current['halId'].astype(str)


# Fusionner les deux DataFrames sur la colonne 'halId'

merged_df = pd.merge(df_previous, df_current, on='halId', how='outer', indicator=True)


# Identifier les enregistrements qui sont uniquement dans df_previous et donc qui ont disparu le jour suivant
df_previous_only = merged_df[merged_df['_merge'] == 'left_only'].copy()


# Identifier les enregistrements qui sont uniquement dans df_current et donc qui sont apparues depuis la veille
df_current_only = merged_df[merged_df['_merge'] == 'right_only'].copy()

# Pour fichier de log, indication des résultats de la comparaison
logging.info(f"Nombre d'enregistrements supprimés : {len(df_previous_only)}")
logging.info(f"Nombre d'enregistrements nouveaux : {len(df_current_only)}")

df_previous_only.head()

étape de comparaison des listes quotidiennes


,docID_x,halId,uri_x,submitType_s_x,docID_y,uri_y,submitType_s_y,_merge
30188,1000117,halshs-01000117,https://shs.hal.science/halshs-01000117v1,file,NaN,NaN,NaN,left_only


In [4]:
df_current_only.head()

,docID_x,halId,uri_x,submitType_s_x,docID_y,uri_y,submitType_s_y,_merge


In [5]:

#suppression de la colonne de résultat de comparaison (_merge : both, left-only, right-only)
df_previous_only = df_previous_only.drop(columns=['_merge'])
df_current_only = df_current_only.drop(columns=['_merge'])

# Renommage des colonnes
df_previous_only = df_previous_only.rename(columns={'docID_x': 'docID', 'halId_x': 'halId', 'uri_x': 'uri', 'submitType_s_x': 'submitType_s'})
df_current_only = df_current_only.rename(columns={'docID_x': 'docID', 'uri_x': 'uri', 'submitType_s_x': 'submitType_s'})

# On ne garde que les colonnes qui nous intéressent
df_previous_only = df_previous_only[['docID', 'halId', 'uri', 'submitType_s']]
df_current_only = df_current_only[['docID', 'halId', 'uri', 'submitType_s']]


# Enregistrer les références des notices supprimées dans un fichier CSV
fichier_suppr = os.path.join(suppressed_folder,f'{date_extraction_current}_fichiers_a_supprimer.csv')

df_previous_only.to_csv(fichier_suppr, index=False)

# Si nécessaire, on peut enregistrer la liste des nouvelles notices en décommentant la ligne suivante:
#df_current_only.to_csv('notices_nouvelles.csv', index=False)

# Message indiquant la fin du traitement
print(f"{fichier_suppr} enregistré.")
logging.info(f"{fichier_suppr} enregistré.")



/data/Suppressions\2024-12-16_fichiers_a_supprimer.csv enregistré.


In [6]:
##########################################################################################
# Suppression des notices identifées comme supprimées 
# Suppression des fichiers correspondants
# méthode : on supprime tout fichier dont le nom contient le halID concerné
##########################################################################################

import pandas as pd
import os
import re

print(f"***étape de suppressions***")

# Lire le fichier des notices à supprimer dans un dataframe
df_suppr = pd.read_csv(fichier_suppr)

# Vérifier si le DataFrame est vide, si oui, on s'arrête.
if df_suppr.empty:
    logging.info(f"Fichier de notices à supprimer vide")
    exit

# Répertoire contenant les fichiers PDF
pdf_directory = '/data/pdf_de_hal'
# Répertoire contenant les fichiers xml
xml_directory = '/data/Notices_Xml-tei_de_Hal'


################
## Suppression des fichiers attachés
################
# Lire les valeurs de halId des notices et fichiers à supprimer
hal_ids_to_delete = df_suppr['halId'].tolist()

# Lister tous les fichiers dans le répertoire
for filename in os.listdir(pdf_directory):
    # Vérifier si le nom de fichier contient l'une des valeurs de halId
    if any(hal_id in filename for hal_id in hal_ids_to_delete):
        file_path = os.path.join(pdf_directory, filename)
        try:
            # Supprimer le fichier
            os.remove(file_path)
            logging.info(f"Supprimé : {file_path}")
        except Exception as e:
            logging.info(f"Erreur lors de la suppression de {file_path}: {e}")


################
## Suppression des notices XML
################
# Lister tous les fichiers dans le répertoire
for filename in os.listdir(xml_directory):
    # Vérifier si le nom de fichier contient l'une des valeurs de halId
    if any(hal_id in filename for hal_id in hal_ids_to_delete):
        file_path = os.path.join(xml_directory, filename)
        try:
            # Supprimer le fichier
            os.remove(file_path)
            logging.info(f"Supprimé : {file_path}")
        except Exception as e:
            logging.info(f"Erreur lors de la suppression de {file_path}: {e}")

print("Terminé")

# Appeler shutdown pour fermer les handlers du fichier de log
logging.shutdown()

***étape de suppressions***
Terminé
